# End-to-End Multi-Class Dog Breed Classification

Using TF 2.0 for multi class classification.

## 1. Problem
>Identifying the breed of a given dog by its image.

## 2. Data
Data from Kaggle competition : https://www.kaggle.com/competitions/dog-breed-identification/data

## 3. Evaluation
According to Kaggle :
>Submissions are evaluated on Multi Class Log Loss between the predicted probability and the observed target.

## 4. Features
* Our data is in form of images (unstructed data)
* We have 120 dog breed (120 class)
* Labels can be found on `labels.csv`
* Both training (with labaels) and test (without labels) sets have +10k image each

In [ ]:
# Setup to import data directly from Kaggle
# Make sure to generate api key from your Kaggle account
# import the json file here then run the following commands
# uncomment the following to run

# ! pip install kaggle

# ! pip install kaggle

# ! cp kaggle.json ~/.kaggle/

# ! chmod 600 ~/.kaggle/kaggle.json

# ! kaggle competitions download -c dog-breed-identification

# ! unzip dog-breed-identification.zip -d data